# Support Vector Machine with Heart DAta Set

### Dataset link
https://www.kaggle.com/cherngs/heart-disease-cleveland-uci

In [31]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.metrics import classification_report

#for time count we use belwo library
from timeit import default_timer as timer
from datetime import timedelta

 yi * (W) power (t) Xi  + bi >= 1


(w*, b*) = ( ||w|| /2 ) + Ci + Sumof Zeta : c=how many errors, Zeta = value of the error

## Get the Data

Set index_col=0 to use the first column as the index.

In [32]:
df = pd.read_csv("heart_cleveland_upload")

In [33]:
No_of_frauds= len(df[df["condition"]==1])
No_of_normals = len(df[df["condition"]==0])
print("The number of  Class 1s are: ", No_of_frauds)
print("The number of Class 0s are: ", No_of_normals)
total= No_of_frauds + No_of_normals
Fraud_percent= (No_of_frauds / total)*100
Normal_percent= (No_of_normals / total)*100
print("Class 0 percentage = ", Normal_percent)
print("Class 1 percentage = ", Fraud_percent)

The number of  Class 1s are:  137
The number of Class 0s are:  160
Class 0 percentage =  53.87205387205387
Class 1 percentage =  46.12794612794613


In [34]:
df.shape

(297, 14)

In [35]:
df.size

4158

In [36]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


# Standardize the Variables
.

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df.drop('condition',axis=1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [38]:
StandardScaler(copy=True, with_mean=True, with_std=True)
scaled_features = scaler.transform(df.drop('condition',axis=1))

In [39]:
df_feat = pd.DataFrame(scaled_features,columns=df.columns[:-1])
df_feat.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,1.600302,0.691095,-2.240629,1.596266,-0.257179,2.430427,1.010199,-0.812095,-0.696419,-0.820813,0.643781,0.344824,-0.874292
1,1.600302,-1.446980,-2.240629,0.468418,-0.160859,-0.411450,-1.003419,0.061157,-0.696419,0.639470,-0.976583,1.411625,-0.874292
2,1.268242,-1.446980,-2.240629,1.032342,-0.411292,-0.411450,-1.003419,-1.554358,-0.696419,1.326662,2.264145,-0.721976,-0.874292
3,1.157555,0.691095,-2.240629,0.355633,0.667499,2.430427,1.010199,1.065396,-0.696419,0.295874,0.643781,0.344824,-0.874292
4,1.046868,0.691095,-2.240629,-1.223355,-0.700254,-0.411450,1.010199,-0.244481,1.435916,0.639470,0.643781,-0.721976,-0.874292


# Train Test Split

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features,df['condition'],
                                                    test_size=0.30)

# Fitting SVM to the Training set

In [42]:
import time
start = time.time()  

from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

end = time.time()
print('Time of Training the Algorithm in second',end - start)

Time of Training the Algorithm in second 0.4049711227416992


# Predicting the Test set results

In [43]:
y_pred = classifier.predict(X_test)

In [44]:
#accuracy of he model
from sklearn.metrics import accuracy_score
print('Accuracy Score :', accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report
print('\n',classification_report(y_test, y_pred))

Accuracy Score : 0.8111111111111111

               precision    recall  f1-score   support

           0       0.82      0.80      0.81        46
           1       0.80      0.82      0.81        44

    accuracy                           0.81        90
   macro avg       0.81      0.81      0.81        90
weighted avg       0.81      0.81      0.81        90



### Applying Cross Validation to determine accuracy of our model

In [45]:
#Applying 10 fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X=X_train, y = y_train.ravel(), cv = 10)
mean_accuracy= accuracies.mean()*100
std_accuracy= accuracies.std()*100
print("The mean accuracy in %: ", accuracies.mean())
print("The standard deviation in % ", accuracies.std())
print("The accuracy of our model in % is betweeen {} and {}".format(mean_accuracy-std_accuracy, mean_accuracy+std_accuracy))

The mean accuracy in %:  0.8302813852813854
The standard deviation in %  0.07869586046960438
The accuracy of our model in % is betweeen 75.1585524811781 and 90.89772457509898


In [46]:
#Specifying the parameters in dictionaries to try out different parameters.
#The GridSearchCV will try all the parameters and give us the best parameters
from sklearn.model_selection import GridSearchCV


parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train.ravel())
best_accuracy = grid_search.best_score_
print("The best accuracy using gridSearch is", best_accuracy)

best_parameters = grid_search.best_params_
print("The best parameters for using this model is", best_parameters)

The best accuracy using gridSearch is 0.8405797101449275
The best parameters for using this model is {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


C:\Users\Xubair\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### Using Poly Kernel

In [47]:
from sklearn.svm import SVC
classifier= SVC(kernel = 'poly', random_state = 0)  #poly, rbf, sigmoid, precomputed, callable
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

C:\Users\Xubair\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [48]:
#accuracy of he model
from sklearn.metrics import accuracy_score
print('Accuracy Score :', accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report
print('\n\n',classification_report(y_test, y_pred))

Accuracy Score : 0.7555555555555555


               precision    recall  f1-score   support

           0       0.77      0.74      0.76        46
           1       0.74      0.77      0.76        44

    accuracy                           0.76        90
   macro avg       0.76      0.76      0.76        90
weighted avg       0.76      0.76      0.76        90



### Using RBF Kernel

In [49]:
from sklearn.svm import SVC
classifier= SVC(kernel = 'rbf', random_state = 0)  #poly, rbf, sigmoid, precomputed, callable
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

C:\Users\Xubair\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [50]:
#accuracy of he model
from sklearn.metrics import accuracy_score
print('Accuracy Score :', accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report
print('\n\n',classification_report(y_test, y_pred))

Accuracy Score : 0.7666666666666667


               precision    recall  f1-score   support

           0       0.77      0.78      0.77        46
           1       0.77      0.75      0.76        44

    accuracy                           0.77        90
   macro avg       0.77      0.77      0.77        90
weighted avg       0.77      0.77      0.77        90



### Using RBF Kernel

In [51]:
from sklearn.svm import SVC
classifier= SVC(kernel = 'sigmoid', random_state = 0)  #poly, rbf, sigmoid, precomputed, callable
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

C:\Users\Xubair\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [52]:
#accuracy of he model
from sklearn.metrics import accuracy_score
print('Accuracy Score :', accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report
print('\n\n',classification_report(y_test, y_pred))

Accuracy Score : 0.8


               precision    recall  f1-score   support

           0       0.79      0.83      0.81        46
           1       0.81      0.77      0.79        44

    accuracy                           0.80        90
   macro avg       0.80      0.80      0.80        90
weighted avg       0.80      0.80      0.80        90

